#### Generator Function

In [1]:
def countingdown(n):
    print('Counting down from %d' %n)
    while n > 0:
        yield n
        n -= 1
    return     #’return’ here is optional.


In [2]:
c = countingdown(4)
next(c)

Counting down from 4


4

In [3]:
next(c)

3

In [4]:
print(next(c))
print(next(c))
print(next(c))

2
1


StopIteration: 

#### Coroutine

In [14]:
def PrintMatch(matchtext):
    print('looking for', matchtext)
    while True:
        line = (yield)
        if matchtext in line:
            print(line)

matcher = PrintMatch('mayank')
next(matcher)


looking for mayank


In [15]:
matcher.send('afdsfdf')

In [16]:
matcher.send('amayank')

amayank


In [17]:
matcher.close()

Also see the following program which is almost same as above - 

In [18]:
def PrintMatch(matchtext):
    print('looking for', matchtext)
    while True:
        line = (yield 5)
        if matchtext in line:
            print(line)

matcher = PrintMatch('mayank')
next(matcher)


looking for mayank


5

In [19]:
matcher.send('amayank')

amayank


5

Inside a function, the `yield` statement can also be used as an expression that appears on the right side of an assignment operator as shown above. A function that uses `yield` in this manner is known as a coroutine, and it executes in response to values being sent to it. Its behavior is also very similar to a generator. Another more illustrative example – 

In [20]:
def cor(a):
	print('started: a=',a)
	b = yield a
	print('received: b =',b)
	c = yield a+b
	print('received c=',c)


In [21]:
mycor = cor(14)
from inspect import getgeneratorstate
getgeneratorstate(mycor)


'GEN_CREATED'

In [22]:
next(mycor)

started: a= 14


14

In [23]:
getgeneratorstate(mycor)

'GEN_SUSPENDED'

In [24]:
mycor.send(545)

received: b = 545


559

In [25]:
mycor.send(12)

received c= 12


StopIteration: 

In [26]:
getgeneratorstate(mycor)

'GEN_CLOSED'

It’s crucial to understand that the execution of the coroutine is suspended exactly at the `yield` keyword.  One more example – 

In [2]:

>>> def averager():
	total  = 0.0
	count = 0
	average = None
	while True:
		term = yield average
		total += term
		count += 1	
		average = total/count

a = averager()
next(a)


In [3]:
a.send(1)

1.0

In [4]:
a.send(2)

1.5

How `yield` works in expression - 


In [5]:
def test():
    for i in range(1,10):
        x = yield i
        print("x is", x)
        
a =  test()  #generator created
next(a)      #generator started

1

In [6]:
next(a)

x is None


2

In [31]:
a.send(55)

x is 55


2

`next(a)` is equivalent to `a.send(None)`. 

First call to `next(a)` -> `i` is set to `0` in second line. Then, in third line, `yield i` evaluates to `0` which is output. Note that at this moment, generator is suspended so `x` isn't really set to anything. Also, since `next(a)` is equivalent to `a.send(None)`, the value of `x` will be `None` rather than `0`. But this isn't executed until next call to `next(a)`.  

Calling `next(a)` again. Now, `x` is set to `None` from previous execution. (Note that in previous call to `next(a)`, the left part of `x = yield i` was execulted and after this generator was suspended.) After that, `print` command is executed. Uptil now, nothing was `yield`ed. After this, loop continues, `i` is set to `1`, `yield i` is encountered again, which yields `1` and genrator halts at this exact point which won't be resumed until the next call to `next(a)` or `a.send()`

Now when we call `a.send(55)`, `x` is set to `55` which is printed, loop continues setting `i` to `2`. In next line, `yield i` is encountered which yields `2`.



Below is the example from a SO Post. It greatly explain how `yield` work in an expression and also how generator works. 

In [5]:
def gen(n=0):
    while True:
        n = (yield n) or n+1
        

Before running following commands, try to predict the output

In [6]:
g = gen()
print(next(g))
print(next(g))
print(g.send(5))
print(next(g))
print(next(g))

0
1
5
6
7


Following is the explanation by Martjin Pieters - 

"Your confusion lies with `generator.send()`. Sending is *just the same thing as using `next()`*, with the difference being that the `yield` expression produces a different value. Put differently, `next(g)` is the same thing as `g.send(None)`, both operations resume the generator there and then. 

Remember that a generator *starts* paused, at the top. The first `next()` call advances tot he first `yield` expression, stops the generator and then pauses. When a `yield` expression is paused and you call either `next(g)` or `g.send(..)`, the generator is resumed where it is right now, and then runs until the next `yield` expression is reached, at which point it pauses again. 

For your code, this happens:

 - `g` is created,nothing happens in `gen()`
 - `next(g)` actually enters the function body, `n=0` is executed, `yield n` pauses `g` and yields `0`. This is printed.
 - `next(g)` resumes the generator; `None` is returned for `yield n` (nothing was sent after all), so `None or n+1` is executed and `n=1` is set. The loop continues on and `yield n` is reached again, the generator pauses and `1` is yielded. This is printed.
 - `g.send(5)` resumes the generator. `5 or n+1` means `n = 5` is executed. The loop continues until `yield n` is reached, the generator is paused, `5` is yielded and you print `5`.
 - `next(g)` resumes the generator, `None` is returned (nothing was sent again), so `None or n+1` is executed and `n=6` is set. The loop continues on and `yield n` is reached again, the generator pauses and `6` is yielded printed.
 - `next(g)` resumes the generator; `None` is returned (nothing was sent again), so `None or n+1` is executed and `n =  7` is set. The loop continues on and `yield n` is reached again, the generator pauses and `7` is yielded and printed.
 
Given your steps 1, 2 and 3, the acutal order is 3, 2, 1 then, with the addition that `next()` also goes through step 2, producing `None`, and 1 being the *next* invocation of `yield` encountered after un-pausing. " 

**--end of SO Post--**

The following part is reproduced here from the notebook `Concurrency in Python`. Originally, this is also from a SO post. 


##### Generators

Generators are objects that allow us to suspend the execution of a Python function. User curated generators are implemented using th keyword `yield`. By creating a normal function containing the `yield` keyword, we turn that function into a generator:


In [31]:
def test():
    yield 1
    yield 2
    
gen = test()
next(gen)

1

In [32]:
next(gen)

2

In [33]:
next(gen)

StopIteration: 

As you can see, calling `next()` on the generator causes the interpreter to load test's frame, and return the `yield`ed value. Calling `next()` again, cause the frame to load again into the interpreter stack, and continue on `yield`ing another value.

By the third time `next()` is called, our generator was finished, and `StopIteration` was thrown.

##### Communicating with a generator

A less known feature of generators, is the fact that you can communicate with them using two methods: `send()` and `throw()`.

In [34]:
def test():
    val = yield 1
    print(val)
    yield 2
    yield 3
    
gen =  test()
next(gen)

1

In [35]:
gen.send('abc')

abc


2

In [36]:
gen.throw(Exception())

Exception: 

Upon calling `gen.send()`, the value is passed as a return value from the `yield` keyword.

`gen.throw()` on the other hand allows throwing Exceptions inside generators, with the exception raised at the same spot `yield` was called.

##### Returning values from generators

Returning a value from a generator, results in the value being put inside the `StopIteration` exception, we can later on recover the value from the exception and use it to our need.

In [37]:
def test():
    yield 1
    return 'abc'

gen = test()
next(gen)

1

In [38]:
try:
    next(gen)
except StopIteration as exc:
    print(exc.value)

abc


##### Behold, a new keyword: `yield from`

Python 3.4 came with the addition of a new keyword: `yield from`. What that keyword allows us to do, is pass on any `next()`, `send()` and `throw()` into an inner-most nested generator. If the inner generators return a value, it is also the return value of `yield from`:

In [35]:
def inner():
    print((yield 2))
    return 33

def outer():
    yield 1
    val = yield from inner()
    print("val is: ",val)
    yield 4
    
gen = outer()
next(gen)

1

In [36]:
next(gen)

2

In [37]:
gen.send('abc')

abc
val is:  33


4